In [0]:
%fs ls dbfs:/FileStore/tables/

In [0]:
SELECT * FROM binaryFile.`dbfs:/FileStore/tables/`

In [0]:
SELECT * FROM text.`dbfs:/FileStore/tables/`

In [0]:
%fs ls dbfs:/FileStore/tables/

In [0]:
select * from CSV.`dbfs:/FileStore/tables/machine_readable_business_employment_data_sep_2024_quarter.csv`

In [0]:
create table csv_data as
(select * from CSV.`dbfs:/FileStore/tables/machine_readable_business_employment_data_sep_2024_quarter.csv`);

In [0]:
select * from csv_data;

In [0]:
create table csv_2 
(Series_reference string, Data_value double, Magnitude int, Group string)
using csv
options(
        path = 'dbfs:/FileStore/tables/machine_readable_business_employment_data_sep_2024_quarter.csv',
        header = 'true',
        delimiter = ',')

In [0]:
create table csv_books
(Title string, Author string, Genre string, Height int, Publisher string)
using csv
options(
        path = 'dbfs:/FileStore/tables/books.csv',
        header = 'true',
        delimiter = ',')


In [0]:
describe extended csv_books;

In [0]:
select * from csv_books;

In [0]:
select count(*) from csv_books;

In [0]:
%python
files = dbutils.fs.ls(f"dbfs:/FileStore/tables/")
display(files)

In [0]:
-- spark.read
--         .table("csv_books")
--      .write
--       --   .mode("append")
--         .format("csv")
--         .option('header', 'true')
--         .option('delimiter', ',')
--         .save(f"dbfs:/FileStore/tables/books_new.csv")

In [0]:
%python
dbutils.fs.rm(f"dbfs:/FileStore/tables/tables/machine_readable_business_employment_data_sep_2024_quarter.csv", True)

In [0]:
create table csv_data__3 as 
select * from csv.`dbfs:/FileStore/tables/books.csv`

In [0]:
describe extended csv_data__3;

In [0]:
select * from csv_data__3

In [0]:
create temp view csv_demo
(Title string, Author string, Genre string, Height int, Publisher string)
using csv
options(
  path = 'dbfs:/FileStore/tables/books.csv',
  header = 'true',
  delimiter = ','
);

select * from csv_demo

In [0]:
create table csv_view_table as
select * from csv_demo;

In [0]:
select * from csv_view_table;

In [0]:
create temp view csv_demo_2
(Title string, Author string, Genre string, Height int, Publisher string)
using csv
options(
  path = 'dbfs:/FileStore/tables/',
  header = 'true',
  delimiter = ','
);

select * from csv_demo_2

In [0]:
create table csv_view_table_2 as
select * from csv_demo_2;

In [0]:
select count(*) from csv_view_table_2;

In [0]:
show tables;

In [0]:
create or replace table employee(zid string);

In [0]:
describe history employee;

In [0]:
select count(*) from csv_view_table;

In [0]:
insert overwrite csv_view_table
select * from csv_view_table_2;

select count(*) from csv_view_table;

In [0]:
describe history csv_view_table;

In [0]:
insert into csv_view_table
select * from csv_view_table@v0;

In [0]:
select count(*) from csv_view_table;

In [0]:
show tables;

In [0]:
restore table csv_view_table to version as of 0;

In [0]:
select count(*) from csv_view_table;
select count(*) from csv_view_table_2;

In [0]:
merge into csv_view_table cvt
using csv_view_table_2 cvt2
on cvt.Title = cvt2.Title
-- WHEN MATCHED THEN UPDATE SET cvt.title = cvt2.title
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%python
print(23807+211)

In [0]:
%fs ls dbfs:/FileStore/tables/

In [0]:
%python
dbutils.fs.rm('dbfs:/FileStore/tables/sample_file.json')

In [0]:
select * from json.`dbfs:/FileStore/tables/sample_file.json`;

In [0]:
show tables;

In [0]:
%python
# Databricks notebook source
def path_exists(path):
  try:
    dbutils.fs.ls(path)
    return True
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise

# COMMAND ----------

def download_dataset(source, target):
    files = dbutils.fs.ls(source)

    for f in files:
        source_path = f"{source}/{f.name}"
        target_path = f"{target}/{f.name}"
        if not path_exists(target_path):
            print(f"Copying {f.name} ...")
            dbutils.fs.cp(source_path, target_path, True)

# COMMAND ----------

data_source_uri = "s3://dalhussein-courses/datasets/bookstore/v1/"
dataset_bookstore = 'dbfs:/mnt/demo-datasets/bookstore'
data_catalog = 'hive_metastore'
spark.conf.set(f"dataset.bookstore", dataset_bookstore)

# COMMAND ----------

def get_index(dir):
    files = dbutils.fs.ls(dir)
    index = 0
    if files:
        file = max(files).name
        index = int(file.rsplit('.', maxsplit=1)[0])
    return index+1

# COMMAND ----------

def set_current_catalog(catalog_name):
    spark.sql(f"USE CATALOG {catalog_name}")

# COMMAND ----------

# Structured Streaming
streaming_dir = f"{dataset_bookstore}/orders-streaming"
raw_dir = f"{dataset_bookstore}/orders-raw"

def load_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.parquet"
    print(f"Loading {latest_file} file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_dir}/{latest_file}", f"{raw_dir}/{latest_file}")

    
def load_new_data(all=False):
    index = get_index(raw_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_file(index)
            index += 1
    else:
        load_file(index)
        index += 1

# COMMAND ----------

# DLT
streaming_orders_dir = f"{dataset_bookstore}/orders-json-streaming"
streaming_books_dir = f"{dataset_bookstore}/books-streaming"

raw_orders_dir = f"{dataset_bookstore}/orders-json-raw"
raw_books_dir = f"{dataset_bookstore}/books-cdc"

def load_json_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.json"
    print(f"Loading {latest_file} orders file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_orders_dir}/{latest_file}", f"{raw_orders_dir}/{latest_file}")
    print(f"Loading {latest_file} books file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_books_dir}/{latest_file}", f"{raw_books_dir}/{latest_file}")

    
def load_new_json_data(all=False):
    index = get_index(raw_orders_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_json_file(index)
            index += 1
    else:
        load_json_file(index)
        index += 1

# COMMAND ----------

download_dataset(data_source_uri, dataset_bookstore)
set_current_catalog(data_catalog)

In [0]:
%fs ls dbfs:/FileStore/tables

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore`

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-json-streaming/

In [0]:
select * from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`

In [0]:
create or replace temp view customers_data as 
select customer_id, email, profile:first_name, profile:address:country from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`;

In [0]:
select * from customers_data;

In [0]:
select from_json(profile) from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`; 

In [0]:
select profile
from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`
limit 1;

In [0]:
create or replace temp view parsed_cols as 
select customer_id, from_json(profile, schema_of_json('{"first_name":"Dniren","last_name":"Abby","gender":"Female","address":{"street":"768 Mesta Terrace","city":"Annecy","country":"France"}}')) as profile_struct
from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`;

In [0]:
describe parsed_cols;

In [0]:
select customer_id, profile_struct.* from parsed_cols;

In [0]:
select profile_struct.address from parsed_cols
limit 1;

In [0]:
select customer_id, profile_struct.first_name, profile_struct.last_name, profile_struct.gender, profile_struct.address.street, profile_struct.address.city, profile_struct.address.country
from parsed_cols;

In [0]:
create table orders as 
select * from json.`dbfs:/mnt/demo-datasets/bookstore/orders-json-streaming/`;
-- drop table orders;

In [0]:
select * from orders;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/

In [0]:
create or replace temp view orders_temp_1 as 
select customer_id, order_id, explode(books) as book
from orders;

select * from orders_temp_1

In [0]:
select customer_id, 
collect_set(order_id) as orders_set,
collect_set(book.book_id) as books_set
from orders_temp_1
group by customer_id;

In [0]:
select customer_id, 
collect_set(order_id) as orders_set,
array_distinct(collect_set(book.book_id)) as books_set
from orders_temp_1
group by customer_id;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/

In [0]:
select * from csv.`dbfs:/mnt/demo-datasets/bookstore/books-cdc/`;

In [0]:
show tables;

In [0]:
create table books as 
select * from csv.`dbfs:/mnt/demo-datasets/bookstore/books-csv/`;

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore

In [0]:
select * from books;

In [0]:
drop table books

In [0]:
create table books
(book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
using csv
options (
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv/',
  delimiter = ',',
  header = 'true'
)

In [0]:
select * from books;

In [0]:
create temp view books_view
(book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
using csv
options (
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv/',
  delimiter = ';',
  header = 'true'
);

In [0]:
select * from books_view;

In [0]:
create temp view orders_enriched as 
select * from (
  select *, explode(books) as book
  from orders
) o
inner join books_view b 
on o.book.book_id = b.book_id;



In [0]:
select * from orders_enriched;

In [0]:
select book.* from orders_enriched;

In [0]:
create table transaction as
select * from (
  select customer_id,
  book.book_id as book_id,
  book.quantity as quantity
  FROM ORDERS_ENRICHED
) pivot (
  sum(quantity) for book_id in (
    'BO1', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);

In [0]:
select * from transaction;

In [0]:
select distinct book.book_id from orders_enriched;

In [0]:
select distinct book.quantity from orders_enriched;

In [0]:
select order_id, books ,
FILTER(books, i -> i.quantity == 1) as books_quantity_2
from orders;

In [0]:
select order_id, book_quantity_2 
from (
  select order_id, books,
  FILTER(books, i -> i.quantity == 1) as book_quantity_2
  from orders
) where size(book_quantity_2 ) > 0;

In [0]:
select order_id, books,
TRANSFORM(
  books, b -> CAST(b.subtotal*0.8 as int) 
) Discounted_Sub_Total
from orders;
  

In [0]:
select order_id, books,
TRANSFORM (books, b -> cast(b.subtotal*0.5 as int)) as Discounted_Sub_Total
from orders ;

In [0]:
show tables;

In [0]:

%fs ls dbfs:/mnt/demo-datasets/bookstore/

In [0]:
create table customers as 
select * from json.`dbfs:/mnt/demo-datasets/bookstore/customers-json`;

select * from customers;

In [0]:
create or replace function get_url(email string)
returns string

return concat("https://www.",split(email,'@')[1])

In [0]:
select email, get_url(email) as Domain from customers
where email is not null;

In [0]:
describe function get_url;

In [0]:
describe function extended get_url;

In [0]:
create or replace function site_type(email string)
returns string 

return case
      when email like '%.com' then 'Commertial_Business'
      when email like '%.org' then 'Non-profit organizations'
      when email like '%.edu' then 'Educational Purpose'
      else concat('Not known : ', split(email,'@')[1])
      end;


In [0]:
select customer_id, email, site_type(email) as Purpose
from customers
where email is not null;

In [0]:
-- drop function get_url;
-- drop function site_type;